### Aşamalara chart print fonksiyonu koymaca.

In [1]:
import os
import pickle
import itertools
import pandas as pd

from matplotlib import colors
import matplotlib.pyplot as plt

from map_engine.map_generator import map

from quest_engine.quest_generator import *
from quest_engine.freytags_fitness import freytags

from encounter_engine.world_encounters import *

from character_engine.character import *

In [ ]:
#nw = map()
#
#nw.populate_map(n_locations=1024, name="terrain", relaxed=True, k=100)
#nw.populate_map(n_locations=4096, name="civilisation", relaxed=True, k=250)
#nw.populate_map(n_locations=36, name="story_act", relaxed=True, k=250)
#nw.populate_map(n_locations=1024, name="threat", relaxed=True, k=100)
#
#nw.attribute_view(seed_1=24, seed_2=34,
#                  map_name="terrain", view_name="terrain")
#nw.attribute_view(seed_1=98, seed_2=50, map_name="civilisation",
#                  view_name="civilisation", double=False)
#nw.attribute_view(seed_1=34, seed_2=34, map_name="threat",
#                  view_name="threat", double=False)
#nw.attribute_view(seed_1=16, seed_2=16, map_name="story_act",
#                  view_name="story_act", double=False)
#
#nw.attribute_centroids("civilisation")
#nw.attribute_centroids("terrain")
##nw.land_mask()
#
#with open('logs/map_nw', 'wb') as f:
#    pickle.dump(nw, f)

with open("logs/map_nw", "rb") as f:
    nw = pickle.load(f)

nw.land_mask()

In [ ]:
pop = quest_pop(nw,act=2, pop_size=5, steps=4)
ind = pop.evolve(gens=1,early_stop=False, mu_p=.4, xo="pmx", mutation="random_point", fancy_save=True)

cmap = colors.ListedColormap(
    ["blue", "white", "yellowgreen", "khaki", "lawngreen", "slategrey", "darkgreen"])

i = 1
for gen in pop.fancy:
    fig, ax = plt.subplots()
    fig.set_dpi(150)
    fig.set_size_inches(15, 15)
    ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.50)
    for q in gen:
        ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=15)
        ax.plot([i[0] for i in q], [i[1] for i in q], color="red",linestyle="dotted",linewidth=2)
        ax.set_title("Generation: {}".format(i))
    fig.savefig(f"fancy/{i}.png")
    i += 1





In [ ]:
fig, ax = plt.subplots()
fig.set_dpi(150)
fig.set_size_inches(15, 15)
ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.50)

q = ind.path
 
ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=20)
ax.plot([i[0] for i in q], [i[1] for i in q], color="purple",linewidth=3)
ax.set_title("Generation: {}".format(i))
fig.savefig(f"fancy/{i}.png")

In [ ]:
print(ind.path_threat)

ind.check_threat()

In [ ]:

muts = ["complete","single_point","random_point"]
xos = ["pmx","single_point","ar"]
mups =  [.5]

for param in [p for p in itertools.product(*[muts,xos,mups])]:

            params = {
                "map"           :   nw,
                "mutation"      :   param[0],
                "xo"            :   param[1],
                "mu_p"          :   param[2],
                "gens"          :   500,
                "shelf_size"    :   50,
                "pop_size"      :   100,
                "print_it"      :   False,
                "brute_force"   :   False,
                "log"           :   True,
                "early_stop"    :   True,
            }

            ql = quest_library(params)

In [ ]:
params = {
                "map"           :   nw,
                "mutation"      :   "random_point",
                "xo"            :   "pmx",
                "mu_p"          :   .5,
                "gens"          :   500,
                "shelf_size"    :   1,
                "pop_size"      :   100,
                "print_it"      :   True,
                "brute_force"   :   False,
                "log"           :   False,
                "early_stop"    :   False,
            }

a = quest_library(params)

In [ ]:
nw.printer(quest_line = ql_1)

In [ ]:

path = "logs/"
files = os.listdir(path)
files = [f for f in files if f.endswith(".csv")]

df = pd.concat([pd.read_csv(path+f) for f in files])


for f in files:
    os.remove(path+f)
    
df.to_csv("logs/merged.csv", index=False)




In [ ]:
i = 1
fig, ax = plt.subplots()
fig.set_dpi(150)
fig.set_size_inches(15, 15)

cmap = colors.ListedColormap(
    ["blue", "white", "yellowgreen", "khaki", "lawngreen", "slategrey", "darkgreen"])

ax.imshow(nw.views["terrain"].T, cmap=cmap, alpha=.50)



for gen in pop.fancy:
    for q in gen:
        ax.scatter([i[0] for i in q], [i[1] for i in q], color="black",marker="*", s=15)
        ax.plot([i[0] for i in q], [i[1] for i in q], color="red",linestyle="dotted",linewidth=1)
        ax.set_title("Generation: " + str(i))
        fig.savefig(f"fancy/{i}.png")


In [2]:
import imageio
images = []
files = []
for filename in os.listdir("/Users/cetiners/Desktop/Thesis/human_error/fancy"):
    print(filename)
    if ".png" in filename:
        files.append(filename)

files = sorted(files, reverse=True)

for filename in files:
        
    images.append(imageio.imread("fancy/"+filename))
imageio.mimsave('/Users/cetiners/Desktop/Thesis/human_error/fancy/ohhhhyeaaah.gif', images)


88.png
77.png
63.png
62.png
76.png
89.png
60.png
74.png
48.png
49.png
75.png
61.png
59.png
65.png
71.png
.DS_Store
70.png
64.png
58.png
99.png
8.png
72.png
66.png
67.png
73.png
9.png
98.png
101.png
14.png
28.png
29.png
15.png
ohhhhyeaaah.gif
100.png
114.png
102.png
17.png
16.png
103.png
107.png
113.png
12.png
13.png
112.png
106.png
110.png
104.png
39.png
11.png
10.png
38.png
105.png
111.png
108.png
35.png
21.png
20.png
34.png
109.png
22.png
36.png
37.png
23.png
27.png
33.png
32.png
26.png
18.png
30.png
24.png
25.png
31.png
19.png
95.png
81.png
4.png
56.png
42.png
43.png
5.png
57.png
80.png
94.png
82.png
96.png
41.png
55.png
7.png
69.png
68.png
54.png
6.png
40.png
97.png
83.png
87.png
93.png
78.png
44.png
2.png
50.png
3.png
51.png
45.png
79.png
92.png
86.png
90.png
84.png
53.png
1.png
47.png
46.png
52.png
85.png
91.png


In [ ]:
dc = {"a" : 2, "b" : 2}

In [ ]:
nw.meta.keys()

In [ ]:

fig, ax = plt.subplots()
fig.set_dpi(150)
fig.set_size_inches(15, 15)
for a in ['unblurred', 'final']:
    ax.imshow(nw.meta[a].T, alpha=.66)
ax.set_title(a)
#ax.scatter(x, y, c="black", s=15)
#ax.scatter(x_1, y_1, c="red", s=15)